In [21]:
#Importing libraries
import cv2 as cv
import sys
import numpy as np
from PIL import Image

In [22]:
def encodingincrement(bit, value):
    bit = int(bit)
    return bit - value%2

In [23]:
def encodegrayscale(baseimage, targetimage):
    """Function to encrypt a target image inside a base image using lsb method"""
    #Obtaining the dimensions of the base image and the target image
    baseimageheight, baseimagewidth = baseimage.shape
    targetimageheight, targetimagewidth = targetimage.shape
    
    #Initialising the variable for number of bits per pixel
    bitsperpixel = targetimage.itemsize * 8
    
    #Checking if encoding is possible
    if targetimage.size * bitsperpixel > baseimage.size:
        return None
    
    #Creating 1-D array of the target image and base image to make the manipulations easy
    flattenedtargetimage = targetimage.flatten()
    flattenedbaseimage = baseimage.flatten()
    
    #Creating string of bits representing the image to be encoded
    targetstream = ""
    for pixelvalue in flattenedtargetimage:
        targetstream = targetstream + '{0:08b}'.format(pixelvalue)
    
    #Encoding the string of bits in the 1-D array of base image
    for i in range(len(targetstream)):
        flattenedbaseimage[i] += encodingincrement(targetstream[i], flattenedbaseimage[i])
    
    #Reshaping the 1-D array of base image to appropriate dimensions
    encodedimage = flattenedbaseimage.reshape(baseimageheight, baseimagewidth)
    
    return encodedimage
    

In [24]:
def decodegrayscale(encodedimage, targetimageshape):
    """Function to decode the target image from the encoded image"""
    #list of powers of two to convert binary number to int
    bintointconvertor = np.array([2 ** x for x in range(7,-1,-1)])
    
    #List to store the decoded image pixel values
    flatdecodedimage = []
    
    #Number of bits per pixel
    bitsperpixel = 8
    
    #Variable to store single recovered pixel of target image from the encoded image. Initialised to all bits zero
    binarynum = [0] * bitsperpixel
    
    #1-D array storing the encoded image
    flatencodedimage = encodedimage.flatten()
    
    #Variable to store the size of target image
    targetimagesize = targetimageshape[0] * targetimageshape[1]
    
    #Loop iterates for total number of bits to be recovered times
    for i in range(targetimagesize * bitsperpixel + 1):
        #If next set of bitsperpixel number of bits reached
        if (i != 0 and i % bitsperpixel == 0):
            #Append the elementwise multiplication of binary digits and powers of two
            flatdecodedimage.append(sum(np.array(binarynum) * bintointconvertor))
            
        #Recovering the last bit from every pixel of encoded image
        binarynum[i%bitsperpixel] = flatencodedimage[i]%2
        
    #Shaping the 1-D image array to appropriate dimensions    
    decodedimage = (np.array(flatdecodedimage)).reshape(targetimageshape)
    
    return decodedimage

In [25]:
#Reading the images 
targetimagename = input('Enter the path of the image to be encoded')
baseimagename = input('Enter the path of the base image')

#Creating the image objects
targetimage = cv.imread(targetimagename, 0)
#If the image could not be loaded
if targetimage is None:
    print("Error loading image")
    
baseimage = cv.imread(baseimagename, 0)
#If the image could not be loaded
if baseimage is None:
    print("Error loading image")

#Callling the encoding function
encodedimage = encodegrayscale(baseimage, targetimage)

#Checking if the encoding was successful
if encodedimage is None:
    sys.exit("Image to be encoded is too large for the base image")
    
#Using the fromarray function from the PIL library to convert the decodedimage to grayscale image
#np.uint8 converts all value to the range [0,255]
decodedimage = Image.fromarray(np.uint8(decodegrayscale(encodedimage, targetimage.shape)), 'L')

#Saving and then reading the decoded image
decodedimage.save('Decodedimage.png')
img = cv.imread('Decodedimage.png', 0)


#Displaying all 4 images in seperate windows
cv.imshow('Base image', baseimage)
cv.imshow('Target image', targetimage)
cv.imshow('Encoded image', encodedimage)
cv.imshow('Decoded image', img)

#Wait till any key is pressed
cv.waitKey(0)

#Close all windows once any key is pressed
cv.destroyAllWindows()

Enter the path of the image to be encodedstarry_night.jpg
Enter the path of the base imagesunset.jpg
